In [1]:
#restart session after running these, you wont have to run them again
%pip install xformers
%pip install -i https://pypi.org/simple/ bitsandbytes
%pip install accelerate
%pip install peft trl
!pip install "unsloth[colab-new] @ git+https://GitHub.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import pandas as pd

from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
!huggingface-cli login

In [4]:
# load trained model from hugging face
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'sanil41/llama-3-8b-Instruct-bnb-4bit-finance', # Supports Llama, Mistral - replace this!
    load_in_4bit = True,
)

adapter_config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
df_test = pd.read_csv('test_stock_data.csv')
df_train = pd.read_csv('train_stock_data.csv')
df_val = pd.read_csv('val_stock_data.csv')

In [ ]:
df_test.head()

,Company,title,description,Date,Up/Down
0,Apple,"Apple Proved That AI Is a Feature, Not a Product",Other tech companies want to sell you chatbots...,2024-06-13,Up
1,Apple,"Apple Won't Pay for ChatGPT, Will You?",Apple and OpenAI announced a landmark partners...,2024-06-13,Up
2,Apple,The AI upgrade cycle is here,The new Apple Intelligence features coming to ...,2024-06-13,Up
3,Apple,Apple CEO Tim Cook on the Magic Mouse: 'Uhhhhh...,Apple has invented some phenomenal products ov...,2024-06-13,Up
4,Apple,"My favorite iOS 18, iPadOS 18 and watchOS 11 f...",There was so much Apple had to cram into its W...,2024-06-13,Up


In [ ]:
df_test['Company'].unique()

array(['Apple', 'Amazon', 'Tesla', 'Google', 'Disney', 'Netflix',
       'Nvidia'], dtype=object)

In [9]:
# predictions loop

device = 'cuda:0'

results = []
for comp in df_test['Company'].unique():
    df_comp = df_test[df_test['Company'] == comp]
    for date in df_test['Date'].unique():
        f5 = df_comp[df_comp['Date'] == date].iloc[0:15]
        text_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

        You are a helpful AI assistant for stock market tips and recommendations<|eot_id|><|start_header_id|>user<|end_header_id|>

        I am going to provide you with some news blurbs about {comp} from today, and a multiple choice question at the end. Please memorize the news, then answer the question. Here are the blurbs:"""
        for i in range(15):
            text_prompt += f"1) {f5['title'].iloc[i]} : {f5['description'].iloc[i]}\n"
        text_prompt += f"\nNow here is the question: Based on your understanding of the news, determine whether {comp}'s stock will go up or down by open tomorrow. Please respond with either (A) - UP or (B) - DOWN.<|eot_id|><|start_header_id|>assistant<|end_header_id|>The correct answer is \n"
        input_ids = tokenizer(text_prompt, return_tensors='pt').input_ids
        input_ids = input_ids.to(device)
        out = model.generate(input_ids, max_new_tokens=2, do_sample=False, temperature=0)
        pred = tokenizer.decode(out[0])[-10:]
        print(comp, date, pred)
        results.append([comp, date, pred])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Apple 2024-06-13 er is 
(A)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Apple 2024-06-14 er is 
(A)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Nvidia 2024-06-13 er is 
(A)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Nvidia 2024-06-14 er is 
(A)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Nvidia 2024-06-17 er is 
(A)
Nvidia 2024-06-18 er is 
(A)


In [38]:
# Get results to dataframe and sort by Company and then by Date
df_results = pd.DataFrame(results, columns=['Company', 'Date', 'Prediction'])
df_sorted = df_results.sort_values(by=['Company','Date'])

df_sorted.reset_index(drop=True, inplace=True)
df_sorted.head(5)

,Company,Date,Prediction
0,Amazon,2024-06-13,er is \n(B)
1,Amazon,2024-06-14,er is \n(B)
2,Amazon,2024-06-17,er is \n(A)
3,Amazon,2024-06-18,er is \n(B)
4,Apple,2024-06-13,er is \n(A)


In [39]:
# Clean and map the 'Prediction' column. 'A' is mapped to 1, 'B' mapped to 0
df_sorted['Prediction'] = df_sorted['Prediction'].str.strip().str[-2]  #  'A' or 'B' 
df_sorted['Prediction'] = df_sorted['Prediction'].map({'A': 1, 'B': 0})  # Map 'A' to 1 and 'B' to 0
df_sorted.head(10)

,Company,Date,Prediction
0,Amazon,2024-06-13,0
1,Amazon,2024-06-14,0
2,Amazon,2024-06-17,1
3,Amazon,2024-06-18,0
4,Apple,2024-06-13,1
5,Apple,2024-06-14,1
6,Apple,2024-06-17,0
7,Apple,2024-06-18,0
8,Disney,2024-06-13,1
9,Disney,2024-06-14,1


In [40]:
# get actual test labels for Up/Down, Map 'Up' to 1 and 'Down' to 0
grouped_df = df_test.groupby(['Company', 'Date'])['Up/Down'].first().reset_index() # group by company and then by date
test_list = list(grouped_df['Up/Down'])
grouped_df['Up/Down'] = grouped_df['Up/Down'].map({'Up': 1, 'Down': 0}) # map Up/Down to 1/0
grouped_df.head(10)

,Company,Date,Up/Down
0,Amazon,2024-06-13,0
1,Amazon,2024-06-14,0
2,Amazon,2024-06-17,1
3,Amazon,2024-06-18,0
4,Apple,2024-06-13,1
5,Apple,2024-06-14,0
6,Apple,2024-06-17,1
7,Apple,2024-06-18,0
8,Disney,2024-06-13,0
9,Disney,2024-06-14,0


In [44]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

y_pred = list(df_sorted['Prediction'])
y_true = list(grouped_df['Up/Down'])

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)

# Calculate precision, recall, and F1 score
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')

# Display the metrics
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Accuracy: 57.14%
Precision: 0.60
Recall: 0.60
F1 Score: 0.60
